In [1]:
import pandas as pd


In [2]:
# Load the cleaned streaming_titles_info dataset (change the path if necessary)
file_path = r'C:\Users\kimbe\Documents\StreamingAnalysis\data\cleaned_data\streaming_titles_info.csv'
streaming_titles_info = pd.read_csv(file_path)


In [3]:
streaming_titles_info.describe().round(2)

,releaseYear,Amazon,Hulu,Netflix,HBO,Apple,averageRating,numVotes
count,92049.00,92049.00,92049.0,92049.00,92049.00,92049.00,89159.00,89159.00
mean,2007.40,0.68,0.1,0.21,0.07,0.18,6.11,12175.78
std,24.01,0.47,0.3,0.40,0.26,0.39,1.31,66964.57
min,0.00,0.00,0.0,0.00,0.00,0.00,1.00,5.00
25%,2004.00,0.00,0.0,0.00,0.00,0.00,5.30,114.00
50%,2015.00,1.00,0.0,0.00,0.00,0.00,6.30,508.00
75%,2020.00,1.00,0.0,0.00,0.00,0.00,7.10,2594.00
max,2025.00,1.00,1.0,1.00,1.00,1.00,9.90,2970653.00


In [4]:
streaming_titles_info.head(10)

,imdbId,type,genres,releaseYear,title,Amazon,Hulu,Netflix,HBO,Apple,averageRating,numVotes
0,tt0000417,movie,"Action, Adventure, Comedy",1902,A Trip to the Moon,1,0,0,1,1,8.1,57373.0
1,tt0000499,movie,"Action, Adventure, Family",1904,An Impossible Voyage,0,0,0,1,0,7.5,4153.0
2,tt0002646,movie,Drama,1913,Atlantis,1,0,0,0,0,6.5,502.0
3,tt0003014,movie,Drama,1913,Ingeborg Holm,0,0,1,0,0,7.0,1479.0
4,tt0004181,movie,Drama,1914,Judith of Bethulia,1,0,0,0,0,6.2,1482.0
5,tt0004873,movie,"Adventure, Family, Fantasy",1915,Alice in Wonderland,1,0,0,0,0,6.1,805.0
6,tt0004972,movie,"Drama, War",1915,The Birth of a Nation,1,0,0,0,0,6.1,26924.0
7,tt0005078,movie,"Drama, Romance",1915,The Cheat,1,0,0,0,0,6.5,2893.0
8,tt0005302,movie,Drama,1915,"Fanchon, the Cricket",1,0,0,0,0,6.4,366.0
9,tt0005339,movie,Drama,1915,A Fool There Was,1,0,0,0,0,5.7,1080.0


In [5]:
# Check for duplicate imdbId
duplicates = streaming_titles_info[streaming_titles_info.duplicated(subset='imdbId')]
print(f"Number of duplicates: {duplicates.shape[0]}")


Number of duplicates: 0


In [6]:
import numpy as np

# Count missing or infinite values in 'numVotes'
missing_values_count = streaming_titles_info['numVotes'].isna().sum() + (streaming_titles_info['numVotes'] == np.inf).sum() + (streaming_titles_info['numVotes'] == -np.inf).sum()

# Calculate the total number of rows in the dataset
total_rows = streaming_titles_info.shape[0]

# Calculate the percentage of missing or infinite values
missing_percentage = (missing_values_count / total_rows) * 100
print(f"Percentage of missing or infinite values in 'numVotes': {missing_percentage:.2f}%")


Percentage of missing or infinite values in 'numVotes': 3.14%


In [7]:
missing_votes = streaming_titles_info[streaming_titles_info['numVotes'].isna()]
print(f"Rows with missing numVotes: {missing_votes}")


Rows with missing numVotes:           imdbId   type          genres  releaseYear  \
168    tt0021086  movie         Western         1930   
557    tt0028209  movie    Crime, Drama         1936   
936    tt0033202  movie           Drama         1941   
1160   tt0035809  movie  Drama, Romance         1943   
1241   tt0036772  movie           Drama         1944   
...          ...    ...             ...          ...   
91934  tt9866320     tv          Comedy         2019   
91970  tt9886132     tv     Documentary         2016   
91974  tt9887604  movie          Comedy         2021   
91987  tt9892604  movie  Drama, Romance         2021   
92037  tt9906144  movie  Mystery, Short         2019   

                               title  Amazon  Hulu  Netflix  HBO  Apple  \
168               The Lonesome Trail       1     0        0    0      0   
557                      Royal Eagle       1     0        0    0      0   
936                   The Last Fight       1     0        0    0      0   

In [8]:
import numpy as np
import pandas as pd

# Convert 'numVotes' to numeric, coercing errors to NaN, and then convert to int (except for NaN and inf values)
streaming_titles_info['numVotes'] = pd.to_numeric(streaming_titles_info['numVotes'], errors='coerce')

# Keep 'averageRating' as float, coercing errors to NaN (if any)
streaming_titles_info['averageRating'] = pd.to_numeric(streaming_titles_info['averageRating'], errors='coerce')

# Replace infinite values with NaN in both columns
streaming_titles_info.replace([np.inf, -np.inf], np.nan, inplace=True)

# Convert 'numVotes' to integer, handling NaN properly
streaming_titles_info['numVotes'] = streaming_titles_info['numVotes'].fillna(np.nan).astype('Int64')

# Check data types
print(streaming_titles_info[['numVotes', 'averageRating']].dtypes)


numVotes           Int64
averageRating    float64
dtype: object


In [9]:
# Standardize column names (e.g., lowercase and replace spaces with underscores)
streaming_titles_info.columns = streaming_titles_info.columns.str.lower().str.replace(' ', '_')


In [10]:
import pprint

def check_unique_values(df, column):
    unique_counts = df[column].value_counts()
    print(f"Unique values and counts for column '{column}':")
    pprint.pprint(unique_counts.to_dict())


In [11]:
streaming_titles_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92049 entries, 0 to 92048
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   imdbid         92049 non-null  object 
 1   type           92049 non-null  object 
 2   genres         91804 non-null  object 
 3   releaseyear    92049 non-null  int64  
 4   title          92049 non-null  object 
 5   amazon         92049 non-null  int64  
 6   hulu           92049 non-null  int64  
 7   netflix        92049 non-null  int64  
 8   hbo            92049 non-null  int64  
 9   apple          92049 non-null  int64  
 10  averagerating  89159 non-null  float64
 11  numvotes       89159 non-null  Int64  
dtypes: Int64(1), float64(1), int64(6), object(4)
memory usage: 8.5+ MB


In [12]:
check_unique_values(streaming_titles_info, 'releaseyear')

Unique values and counts for column 'releaseyear':
{0: 5,
 1898: 1,
 1901: 1,
 1902: 2,
 1903: 1,
 1904: 1,
 1906: 2,
 1908: 1,
 1909: 1,
 1910: 1,
 1911: 1,
 1913: 2,
 1914: 1,
 1915: 7,
 1916: 5,
 1917: 4,
 1918: 6,
 1919: 10,
 1920: 10,
 1921: 8,
 1922: 6,
 1923: 10,
 1924: 5,
 1925: 17,
 1926: 12,
 1927: 18,
 1928: 15,
 1929: 24,
 1930: 29,
 1931: 39,
 1932: 62,
 1933: 57,
 1934: 80,
 1935: 89,
 1936: 109,
 1937: 108,
 1938: 89,
 1939: 103,
 1940: 82,
 1941: 104,
 1942: 97,
 1943: 98,
 1944: 77,
 1945: 105,
 1946: 113,
 1947: 115,
 1948: 119,
 1949: 114,
 1950: 150,
 1951: 140,
 1952: 166,
 1953: 188,
 1954: 170,
 1955: 146,
 1956: 175,
 1957: 201,
 1958: 211,
 1959: 222,
 1960: 205,
 1961: 204,
 1962: 220,
 1963: 206,
 1964: 240,
 1965: 226,
 1966: 251,
 1967: 252,
 1968: 287,
 1969: 262,
 1970: 277,
 1971: 329,
 1972: 343,
 1973: 319,
 1974: 337,
 1975: 299,
 1976: 342,
 1977: 305,
 1978: 328,
 1979: 357,
 1980: 366,
 1981: 349,
 1982: 356,
 1983: 422,
 1984: 402,
 1985: 415,
 19

In [13]:
# Remove rows where 'releaseYear' is 0 or 2025
cleaned_df = streaming_titles_info[~streaming_titles_info['releaseyear'].isin([0, 2025])]

# Verify the cleaning
print(f"Rows after cleaning: {cleaned_df.shape[0]}")


Rows after cleaning: 92036


In [14]:
# Split genres by comma and remove extra spaces
genres_list = streaming_titles_info['genres'].dropna().apply(lambda x: [genre.strip() for genre in x.split(',')])

# Flatten the list to count unique genres
all_genres = [genre for sublist in genres_list for genre in sublist]

# Create a series with the count of each unique genre
genre_counts = pd.Series(all_genres).value_counts()

# Calculate the percentage of each genre
genre_percentage = (genre_counts / genre_counts.sum()) * 100

# Combine counts and percentages into a DataFrame for better readability
genre_summary = pd.DataFrame({
    'count': genre_counts,
    'percentage': genre_percentage
})

# Sort genres by count in descending order
genre_summary_sorted = genre_summary.sort_values(by='count', ascending=False)

print(genre_summary_sorted)


                    count  percentage
Drama               41057   22.015422
Comedy              25439   13.640800
Action              13679    7.334899
Documentary         11621    6.231366
Romance             11322    6.071038
Crime               11269    6.042618
Thriller            10920    5.855479
Horror               9005    4.828625
Adventure            8377    4.491882
Mystery              6183    3.315424
Animation            6032    3.234455
Family               5296    2.839800
Fantasy              4062    2.178110
Biography            3606    1.933595
Sci-Fi               3270    1.753426
History              3204    1.718036
Music                2324    1.246166
Reality-TV           1730    0.927654
Sport                1575    0.844540
Western              1552    0.832207
War                  1505    0.807005
Short                1433    0.768398
Musical              1146    0.614504
Game-Show             328    0.175879
Film-Noir             195    0.104562
Talk-Show   

In [15]:
# Define the mapping for combining genres as per your request
genre_mapping = {
    'Reality-TV': 'Reality',
    'Reality': 'Reality',
    'Musical': 'Music',
    'Music': 'Music',
    'Game-Show': 'Reality',
    'Film-Noir': 'Crime',
    'Talk-Show': 'News & Talk-Show',
    'News': 'News & Talk-Show',
    'TV Movie': None,  # Drop this genre
    'Adult': None,  # Drop this genre
    'Kids': 'Family',
    'Soap': 'Drama'
}

# Function to apply the mapping to each genre
def group_genre(genre):
    return genre_mapping.get(genre, genre)  # Default to the original genre if not in the mapping

# Apply the genre mapping to the genre list
grouped_genres_list = streaming_titles_info['genres'].dropna().apply(
    lambda x: [group_genre(genre.strip()) for genre in x.split(',')]
)

# Remove None values (this effectively removes 'TV Movie' and 'Adult')
grouped_genres_list = grouped_genres_list.apply(lambda x: [genre for genre in x if genre is not None])

# Flatten the list to count unique grouped genres
all_grouped_genres = [genre for sublist in grouped_genres_list for genre in sublist]

# Create a series with the count of each unique grouped genre
grouped_genre_counts = pd.Series(all_grouped_genres).value_counts()

# Calculate the percentage of each genre
grouped_genre_percentage = (grouped_genre_counts / grouped_genre_counts.sum()) * 100

# Combine counts and percentages into a DataFrame for better readability
grouped_genre_summary = pd.DataFrame({
    'count': grouped_genre_counts,
    'percentage': grouped_genre_percentage
})

# Sort genres by count in descending order
grouped_genre_summary_sorted = grouped_genre_summary.sort_values(by='count', ascending=False)

# Print the result
print(grouped_genre_summary_sorted)

                    count  percentage
Drama               41059   22.020036
Comedy              25439   13.642994
Action              13679    7.336079
Documentary         11621    6.232369
Crime               11464    6.148170
Romance             11322    6.072015
Thriller            10920    5.856421
Horror               9005    4.829402
Adventure            8377    4.492604
Mystery              6183    3.315957
Animation            6032    3.234975
Family               5298    2.841330
Fantasy              4062    2.178460
Biography            3606    1.933906
Music                3470    1.860969
Sci-Fi               3270    1.753709
History              3204    1.718313
Reality              2096    1.124090
Sport                1575    0.844676
Western              1552    0.832341
War                  1505    0.807135
Short                1433    0.768521
News & Talk-Show      285    0.152846
Science Fiction         4    0.002145
Action & Adventure      1    0.000536


In [16]:
# Split the genres and expand into a DataFrame where each genre gets its own row
genres_expanded = streaming_titles_info['genres'].str.split(', ', expand=True)

# Reshape the DataFrame so each genre gets its own row, removing any NaN values
genres_expanded = genres_expanded.melt(value_name='genre').dropna(subset=['genre'])

# One-hot encode the genres by creating a binary column for each genre
one_hot_encoded_genres = pd.get_dummies(genres_expanded['genre'])

# Check the shape of the resulting DataFrame
print(f"Shape of the one-hot encoded genres DataFrame: {one_hot_encoded_genres.shape}")

# Optionally, you can add the one-hot encoded genres back to your original DataFrame
streaming_titles_info = pd.concat([streaming_titles_info, one_hot_encoded_genres], axis=1)


Shape of the one-hot encoded genres DataFrame: (186492, 34)
